*Path of dataset*

In [ ]:
path='../input/traffic-signs/myData'
output_path='./'

In [ ]:
import pickle
import numpy as np


# Libraries

In [ ]:
from os import listdir
from os.path import join
# importing matplotlib modules
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2 as cv
from skimage.feature import greycomatrix, greycoprops

In [ ]:
data=list()
data_names=list()
labeles=list()
for folder in listdir(path):
    for file in listdir(join(path,folder)):
        data_names.append(file)
        labeles.append(int(folder))
labeles=np.array(labeles)        

# Read images

In [ ]:

data_images=list()

    # Read Images

    # i: used to get folder name from labeles
    # file: the name of the image
for i,file in enumerate(data_names):
    image_folder=join(path,str(labeles[i]))
    img = cv.imread(join(image_folder,file))
    data_images.append(img)
data_images=np.array(data_images,dtype=np.uint8)



In [ ]:
plt.imshow(data_images[10])

In [ ]:
data_names[10]

In [ ]:
data_images.shape

In [ ]:
def color_Moments(img):
    R = img[..., 0]
    G = img[..., 1]
    B = img[..., 2]
    colorFeatures = np.zeros((1,7))
    colorFeatures = [np.mean(R),np.std(R),np.mean(G),np.std(G),np.mean(B),np.std(B)]
    colorFeatures /= np.mean(colorFeatures)
    return colorFeatures

In [ ]:
color_Moments(data_images[0])

In [ ]:
def hsvHistogramFeatures(image):
    rows,cols,dd = image.shape
    image = cv.cvtColor(image, cv.COLOR_RGB2HSV)
    
    h = image[...,0]
    s = image[...,1]
    v = image[...,2]

    numberOfLevelsForH = 8
    numberOfLevelsForS = 2
    numberOfLevelsForV = 2

    maxValueForH = np.max(h)
    maxValueForS = np.max(s)
    maxValueForV = np.max(v)

    hsvColorHisto = np.zeros((8,2,2))

    quantizedValueForH = (h*numberOfLevelsForH/maxValueForH)
    quantizedValueForS = (s*numberOfLevelsForS/maxValueForS)
    quantizedValueForV = (v*numberOfLevelsForV/maxValueForV)

    index = np.zeros((rows*cols,3))
    index[:,0] = quantizedValueForH.flatten()
    index[:,1] = quantizedValueForS.flatten()
    index[:,2] = quantizedValueForV.flatten()

    for i in range(len(index[:,0])):
        if(index[i,0]==0 or index[i,1]==0 or index[i,2]==0):
            continue
        hsvColorHisto[int(index[i,0]),int(index[i,1]),int(index[i,2])] +=1

    hsvColorHisto = hsvColorHisto.flatten()
    hsvColorHisto /= np.sum(hsvColorHisto)
    return hsvColorHisto.reshape(-1)

In [ ]:
hsvHistogramFeatures(data_images[0])

In [ ]:
def  textureFeatures(img):
    features_texture = np.zeros((1,4))
    im = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
    gcx = greycomatrix(im,[1],[0, np.pi/2])
    Contrast = greycoprops(gcx,'contrast').mean()
    Correlation = greycoprops(gcx,'correlation').mean()
    Energy = greycoprops(gcx,'energy').mean()
    Homogeneity = greycoprops(gcx,'homogeneity').mean()
    features_texture = [Contrast,Correlation,Energy,Homogeneity]
    return features_texture/np.sum(features_texture)

In [ ]:
textureFeatures(data_images[0])

In [ ]:
def shapeFeatures(img):
    im = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
    shapeFeat = cv.HuMoments(cv.moments(im)).flatten()
    shapeFeat /= np.mean(shapeFeat)
    return shapeFeat

In [ ]:
shapeFeatures(data_images[0])

In [ ]:
def getFeatures(img):
    features = np.concatenate((color_Moments(img),shapeFeatures(img),textureFeatures(img)))
    return features

In [ ]:
getFeatures(data_images[0]).shape

In [ ]:
data_images.shape[0]

In [ ]:
def CBIR_Indexation(data_images):
    length=data_images.shape[0]
    features_data=list()
    print('Debut d''Indexation')
    for i in range (length):
        features_data.append(getFeatures(data_images[i]))
    print('Fin d''Indexation');
    return np.array(features_data)

## Edge detection

In [ ]:
from skimage.io import imread, imshow
from skimage.filters import prewitt_h,prewitt_v
def get_edge_detection(image):
    #reading the image 
    image =cv.cvtColor(data_images[10],cv.COLOR_BGR2GRAY)  #cv.imread('../input/traffic-signs/myData/0/0_10000_1577671998.6491628.png',cv.IMREAD_GRAYSCALE)

    #calculating horizontal edges using prewitt kernel
    edges_prewitt_horizontal = prewitt_h(image)
    #calculating vertical edges using prewitt kernel
    edges_prewitt_vertical = prewitt_v(image)
    return edges_prewitt_horizontal+edges_prewitt_vertical


## Matrix of all images of edge detection

In [ ]:
# matrix_edges=[]
# for image in data_images:
#     edges=get_edge_detection(image)
#     matrix_edges.append(edges)
# matrix_edges=np.array(matrix_edges)

In [ ]:
# plt.imshow(matrix_edges[0])

### reshape image to one vectors decsriptor

In [ ]:
# matrix_edges[0].reshape(-1)

In [ ]:
# matrix_edges_vectors=np.array([image.reshape(-1) for image in matrix_edges])

In [ ]:
# matrix_edges_vectors.shape

# CBIR

In [ ]:
data_base_index=CBIR_Indexation(data_images)

## Save data

In [ ]:
with open('./cbir_base_index.pickle','wb') as file:
    pickle.dump([data_base_index,labeles],file)

# Machine learning Models

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
x_train_o,x_test_o,y_train,y_test=train_test_split(data_base_index,labeles,test_size=0.20, random_state=200)

# we will use this to avoid negative and zero value
max=np.amax(x_train_o)
min=np.amin(x_train_o)
#safe=(max+min)+1
safe=max

In [ ]:
data_base_index.shape

In [ ]:
safe

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

pca = PCA(n_components=14)
x_train=pca.fit_transform(x_train_o)
x_test=pca.transform(x_test_o)

# clf = RandomForestClassifier(random_state=0)
# print(clf.fit(x_train, y_train).score(x_test,y_test))


# Data transformation

We are going to use this function to show all scores with # transformation

In [ ]:
# fucntion to get time of execution
import time
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import NuSVC
from sklearn.ensemble import GradientBoostingClassifier
def get_max(scores):
    max=[]
    for item in scores:
        max.append(item[1])
    return sorted(max)[-1]    
clf=RandomForestClassifier(random_state=0)
def classifier(x_train,x_test,y_train=y_train,y_test=y_test):
    scores=[]
    #start_time = time.time()
    scores.append(('RandomForest',clf.fit(x_train,y_train).score(x_test,y_test)))
    print(scores)
    score=clf.score(x_test,y_test)
    clf = BernoulliNB()
    scores.append(('BernoulliNB',clf.fit(x_train,y_train).score(x_test,y_test)))
    print(get_max(scores))
    
    clf = DecisionTreeClassifier(random_state=0)
    scores.append(('DecisionTreeClassifier',clf.fit(x_train,y_train).score(x_test,y_test)))
    print(get_max(scores))
    
    extra_tree = ExtraTreeClassifier(random_state=0)
    clf = BaggingClassifier(extra_tree, random_state=0)
    scores.append(('ExtraTreeClassifier',clf.fit(x_train,y_train).score(x_test,y_test)))
    print(get_max(scores))
    
    clf = GaussianNB()
    scores.append(('GaussianNB',clf.fit(x_train,y_train).score(x_test,y_test)))
    print(get_max(scores))
    
    clf = KNeighborsClassifier(n_neighbors=3)
    scores.append(('KNeighborsClassifier',clf.fit(x_train,y_train).score(x_test,y_test)))
    print(get_max(scores))
    
    clf = MLPClassifier(random_state=1, max_iter=300)
    scores.append(('MLPClassifier',clf.fit(x_train,y_train).score(x_test,y_test)))
    print(get_max(scores))
    
    clf = RidgeClassifier()
    scores.append(('RidgeClassifier',clf.fit(x_train,y_train).score(x_test,y_test)))
    print(get_max(scores))
    
    clf=NuSVC(nu=0.1)
    scores.append(('NuSVC',clf.fit(x_train,y_train).score(x_test,y_test)))
    print(get_max(scores))
    
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
    scores.append(('GradientBoostingClassifier',clf.fit(x_train,y_train).score(x_test,y_test)))
    print(get_max(scores))
    
    return scores,'fin'

## Brut

In [ ]:
score,t=classifier(x_train,x_test)
print(score)

## LOG

In [ ]:
x_train_l=np.log2(x_train+safe)
x_test_l=np.log2(x_test+safe)
score,t=classifier(x_train_l,x_test_l)
print(score)

## ZScore

In [ ]:
from scipy.stats import zscore
x_train_zs=zscore(x_train)
x_test_zs=zscore(x_test)
score,t=classifier(x_train_zs,x_test_zs)
print(score)

## Min-Max normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_n=scaler.transform(x_train)
x_test_n=scaler.transform(x_test)
score,t=classifier(x_train_n,x_test_n)
print(score)


## Log + standardisation (zscore)

In [ ]:
x_train_l_zs=zscore(x_train_l)
x_test_l_zs=zscore(x_test_l)
score,t=classifier(x_train_l_zs,x_test_l_zs)
print(score)


## Remove low variance feature

In [ ]:
from sklearn.feature_selection import VarianceThreshold
variance=VarianceThreshold()
x_train_low=variance.fit_transform(x_train,y_train)
x_test_low=variance.transform(x_test)
score,t=classifier(x_train_low,x_test_low)
print(score)

## Remove low variance feature + TL + Normalisation

In [ ]:
#normalisation
scaler = MinMaxScaler()
 
x_train_l_n_low=variance.fit_transform(scaler.fit_transform(x_train_l),y_train)

x_test_l_n_low=variance.transform(scaler.transform(x_test_l))

score,t=classifier(x_train_l_n_low,x_test_l_n_low)
print(score)

## Chi2

In [ ]:
from sklearn.feature_selection import SelectKBest,chi2
k = SelectKBest(chi2, k=13)
x_train_chi2=k.fit_transform(x_train+safe,y_train)
x_test_chi2 = k.transform(x_test+safe)
score,t=classifier(x_train_chi2,x_test_chi2)
print(score)

## RFE

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
estimator = SVR(kernel="linear")

selector = RFE(estimator, n_features_to_select=6, step=1)
x_train_rfe=selector.fit_transform(x_train,y_train)
x_test_rfe=selector.transform(x_test)
print('classifier')
score,t=classifier(x_train_rfe,x_test_rfe)
print(score)

## Relief

In [ ]:
!pip install sklearn-relief==1.0.0b2

In [ ]:
from sklearn_relief import Relief
r=Relief(n_features=10)

x_train_relief=r.fit_transform(x_train,y_train)
x_test_relief=r.transform(x_test)

score,t=classifier(x_train_relief,x_test_relief)
print(score)

## Chi2 + TL + Normalisation (13 features)

In [ ]:

k = SelectKBest(chi2, k=13)
x_train_l_n_chi2=k.fit_transform(scaler.fit_transform(x_train_l),y_train)
x_test_l_n_chi2 = k.transform(scaler.transform(x_test_l))
score,t=classifier(x_train_l_n_chi2,x_test_l_n_chi2)
print(score)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
gc = RandomizedSearchCV(estimator=clf,param_distributions =random_grid)
gc.fit(x_train_chi2,y_train)

In [ ]:
# summarize results
print("Best: %f using %s" % (gc.best_score_, gc.best_params_))


In [ ]:
clf.set_params(**gc.best_params_)

In [ ]:
clf.score(x_test_chi2,y_test)

In [ ]:
gc.best_estimator_